In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch import Tensor
from torch.nn.utils.rnn import (
    pack_sequence,
    pad_sequence,
    pack_padded_sequence,
    pad_packed_sequence,
    PackedSequence,
)
import xarray

In [ ]:
import datetime
from typing import Final, Type, Union

import numpy as np
from numpy import datetime64, ndarray, timedelta64
from numpy.typing import ArrayLike, NDArray
import pandas
from pandas import Timedelta, Timestamp

TimeStampLike = Union[str, datetime.datetime, np.datetime64, pandas.Timestamp]
TimeDeltaLike = Union[str, datetime.timedelta, np.timedelta64, pandas.Timedelta]

In [ ]:
import tsdm
from tsdm.datasets import InSilicoData

In [ ]:
tsdm.random.sample_timestamps(size=10, freq="15s")

In [ ]:
class VaryingLengthDataset(Dataset):
    def __init__(self, n, dmin, dmax):
        super().__init__()
        self.size = n
        self.data = [
            torch.randint(0, 10, (np.random.randint(dmin, dmax), 2)) for _ in range(n)
        ]

    # a commemt
    def __len__(self):
        return self.size

    def __getitem__(self, index):
        return self.data[index]

    def __repr__(self):
        return "".join(repr(x.shape) for x in self.data)


def myfun():
    pass

In [ ]:
dsets = []

for key, df in InSilicoData.dataset.items():
    ds = df.to_xarray()
    ds = ds.assign_coords(run_id=key)
    dsets.append(ds)

In [ ]:
xarray.concat(dsets, dim="run_id").to_dataframe()

In [ ]:
dsets

In [ ]:
arrays = [xarray.DataArray(df, name=key) for key, df in InSilicoData.dataset.items()]

In [ ]:
arrays[0]

In [ ]:
xarray.merge(arrays)

### Batching this Dataset won't work:

In [ ]:
MyDataset = VaryingLengthDataset(100, 3, 7)

try:
    next(iter(DataLoader(MyDataset, batch_size=10)))
    print(x)
except Exception as E:
    print(E)

### Let's try with a custom collate_fn

In [ ]:
np.random.randint(0, len(MyDataset), 20)

In [ ]:
batch = [MyDataset[idx] for idx in np.random.randint(0, len(MyDataset), 5)]
batch

In [ ]:
pad_sequence(batch, batch_first=True).shape

In [ ]:
def collate_list(batch):
    return batch


try:
    x = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_list)))
    print(x)
except Exception as E:
    print(E)

In [ ]:
def collate_packed(batch):
    batch.sort(key=torch.Tensor.__len__, reverse=True)
    return pack_sequence(batch)


try:
    x = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_packed)))
    print(x)
except Exception as E:
    print(E)

In [ ]:
batch = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_list)))
[x.T for x in batch]

In [ ]:
batch_sorted = batch.copy()
batch_sorted.sort(key=torch.Tensor.__len__, reverse=True)
[x.T for x in batch_sorted]

In [ ]:
batch_packed = pack_sequence(batch_sorted)
batch_packed.data.T, batch_packed.batch_sizes

In [ ]:
batch_pad_packed, lengths = pad_packed_sequence(batch_packed, batch_first=True)
torch.swapaxes(batch_pad_packed, 1, 2), lengths

In [ ]:
unpacked_batch = [x[:l].T for x, l in zip(batch_pad_packed, lengths)]
unpacked_batch

In [ ]:
def reconstruct_batch(packed: PackedSequence) -> list[Tensor]:
    d = packed.data.shape[-1]
    dtype = packed.data.dtype
    device = packed.data.device

    lengths = []
    b0 = 0
    for b in batch_sizes:

        lengths += [b]

In [ ]:
reconstructed_batch = batch_pad_packed
reconstructed_batch

In [ ]:
batch_pad_packed.shape

In [ ]:
def unpack(packed):
    pass

In [ ]:
a = MyDataset[:5]
a.sort(key=lambda x: len(x))
a